### Importing Libraries:

In [3]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
from numpy import log
from itertools import combinations
import yfinance as yf

### Initialization:

In [4]:
VERBOSE = False

# --------------------- CoinMarketCap: get top coins ---------------------
API_KEY = "13e10784c3884bb8b374a661da8b4631"  # keep this secret in real projects
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY}
params = {"start": "1", "limit": "70", "convert": "USD"}


# request the top coins by market cap
response = requests.get(url, headers=headers, params=params)
data = response.json()["data"]
# build a small DataFrame with symbols only
top70_df = pd.DataFrame([{"symbol": coin["symbol"]} for coin in data])

#  backtest / history parameters
# Backtest start and end (tz-aware UTC timestamps)
BACKTEST_START = pd.Timestamp("2025-01-01", tz="UTC")
BACKTEST_END   = pd.Timestamp("2025-12-31", tz="UTC")
HISTORY_START  = "2023-01-01"   # earliest date to fetch history (ensures enough pre-backtest data)
TARGET_N = 50                   # target number of assets to select

# --------------------- Phase 1: Universe selection by EMA200 ratio ---------------------
selected = []   # list of selected tickers (pass EMA filter)
main_dict = {}  # store full historical df for each selected ticker (used later for backtest)
crypto_pointer = 0

# iterate through top coins until we select TARGET_N assets or exhaust the list
while len(selected) < TARGET_N and crypto_pointer < len(top70_df):
    symbol = top70_df.iloc[crypto_pointer]["symbol"]
    crypto_pointer += 1
    ticker = symbol + "-USD"

    # fetch daily historical OHLC data (include time through backtest end)
    hist = yf.Ticker(ticker).history(
        interval="1d",
        start=HISTORY_START,
        end=(BACKTEST_END + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    )

    # skip tickers with no usable price data
    if hist.empty or hist["Close"].isna().all():
        continue

    # keep only the needed columns and convert index to a timestamp column
    df = hist[["Open", "High", "Low", "Close"]].reset_index()
    df.rename(columns={"Date": "timestamp"}, inplace=True)
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    # ensure all timestamps are timezone-aware and in UTC to avoid tz comparison errors
    if df["timestamp"].dt.tz is None:
        df["timestamp"] = df["timestamp"].dt.tz_localize("UTC")
    else:
        df["timestamp"] = df["timestamp"].dt.tz_convert("UTC")



    # ticker passed the EMA filter -> store full df (we will slice BACKTEST range at backtest time)
    selected.append(ticker)
    main_dict[ticker] = df

# print selected tickers summary
if VERBOSE:
    print("Selected count:", len(selected))
    print(selected)
    print("------------------")

# --------------------- applying EMA200 filter----------------------
ema200_filtered_tickers = []
for ticker in selected:
    df = main_dict[ticker]
    # build the pre-backtest slice (use only data strictly before BACKTEST_START)
    pre_df = df.loc[df["timestamp"] < BACKTEST_START].copy()
    # require at least 200 historical candles to compute a reliable EMA200
    if len(pre_df) < 200:
        continue

    # compute EMA200 on pre-backtest data only (avoids lookahead)
    pre_df.loc[:, "EMA200"] = pre_df["Close"].ewm(span=200, adjust=False).mean()

    # compute the fraction of pre-backtest closes that are above EMA200
    ratio_above = (pre_df["Close"] > pre_df["EMA200"]).mean()

    # apply the >60% rule: keep the ticker only if more than half of closes were above EMA200
    if ratio_above <= 0.6:
        continue
    ema200_filtered_tickers.append(ticker)
if VERBOSE:
    print("filtered by ema tickers:", len(ema200_filtered_tickers))
    print(ema200_filtered_tickers)

# --------------------- ATR-based grid spacing ---------------------
ATR_PERIOD = 14
GRID_MULTIPLIER = 1.0   # spacing = GRID_MULTIPLIER * ATR
MIN_HISTORY_FOR_ATR = ATR_PERIOD

spacing_meta = {}  # will hold last_atr, spacing and spacing% for each ticker

def compute_atr_series(df, period=14):
    """
    Compute ATR series for a DataFrame with 'High','Low','Close'.
    Returns a pandas Series of the ATR (NaN for the first `period-1` rows).
    """
    high = df["High"]
    low = df["Low"]
    close = df["Close"]
    prev_close = close.shift(1)

    # True Range components and the TR per row
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)

    # ATR as a simple rolling mean of TR (min_periods=period => first valid after `period` rows)
    atr = tr.rolling(window=period, min_periods=period).mean()
    return atr



# compute ATR and derive grid spacing for each selected ticker
for ticker in ema200_filtered_tickers:
    df_all = main_dict[ticker]
    df = df_all.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])  # ensure datetime

    # use only data before the backtest start to compute ATR (no lookahead)
    pre_df = df.loc[df["timestamp"] < BACKTEST_START].copy()
    if len(pre_df) < MIN_HISTORY_FOR_ATR:
        # not enough data to compute ATR(period)
        continue

    # compute ATR series on pre-backtest data
    pre_df.loc[:, "ATR"] = compute_atr_series(pre_df, period=ATR_PERIOD)

    # take the last ATR value as representative recent volatility
    last_atr = pre_df["ATR"].iloc[-1]
    if pd.isna(last_atr):
        # ATR may be NaN if there weren't enough TR values; skip in that case
        continue

    # grid spacing in price units (e.g., USD)
    grid_spacing = GRID_MULTIPLIER * last_atr

    # also compute spacing as a percent of last close for convenience
    last_close = pre_df["Close"].iloc[-1]
    grid_spacing_pct = grid_spacing / last_close if last_close != 0 else None

    spacing_meta[ticker] = {
        "last_atr": float(last_atr),
        "grid_spacing": float(grid_spacing),
        "grid_spacing_pct": float(grid_spacing_pct) if grid_spacing_pct is not None else None,
        "last_close": float(last_close)
    }
if VERBOSE:
    print("------------------")
    # print spacing summary for each ticker
    for t, meta in spacing_meta.items():
        print(f"{t}: ATR={meta['last_atr']:.4f}, spacing={meta['grid_spacing']:.4f}, spacing%={meta['grid_spacing_pct']*100:.2f}%")


$TAO-USD: possibly delisted; no price data found  (1d 2023-01-01 -> 2026-01-01)
$PEPE-USD: possibly delisted; no price data found  (1d 2023-01-01 -> 2026-01-01)


### Grid Configuration:

In [5]:
def create_grid_levels(current_price, grid_spacing, n_levels=8):
    """
    Create grid levels below the current market price.
    """
    buy_levels = np.array([current_price - (i + 1) * grid_spacing for i in range(n_levels)])
    if VERBOSE:
        print(f"this is the buy levels: {buy_levels}" )
    sell_levels = buy_levels + grid_spacing  # Sell level is 1 grid step above buy level
    if VERBOSE:
        print(f"this is the sell levels: {sell_levels}" )
    return buy_levels, sell_levels



def compute_martingale_sizes(base_size, multiplier, n_levels):
    """
    Compute exponentially increasing position sizes for grid levels.
    """
    sizes = []

    for i in range(n_levels):
        size = base_size * (multiplier ** i)
        sizes.append(size)
    if VERBOSE:
        print(f"this is the position sizes: {sizes}")
    return sizes


# Just for testing the compute_average_price method and it is not used for strategy implementation.
filled_buys = [
    {"buy_price": 100, "size": 1},
    {"buy_price": 90,  "size": 2},
    {"buy_price": 80,  "size": 4},
]


def compute_average_price(positions):
    """
    Compute weighted average buy price of ACTIVE (open) positions.
    Expects each position dict to have keys: 'buy' and 'size'.
    """
    total_cost = 0.0
    total_size = 0.0

    for p in positions:
        total_cost += p["buy"] * p["size"]
        total_size += p["size"]

    if total_size == 0:
        return None

    return total_cost / total_size




### Martingale Grid (Long Only):

In [ ]:
#  Phase 3 : monthly multi-timeframe grid runner 

# Parameters:
N_LEVELS = 10
USE_BUY_LEVELS = 8
BASE_SIZE = 1.5
MARTINGALE_MULTIPLIER = 2.0
TAKE_PROFIT_PCT = 0.025
STOP_LOSS_INDEX = N_LEVELS - 1

TIMEFRAMES = ["1d", "4h", "1h"]  

#  helper method:
def initialize_grid_positions(buy_levels, sell_levels, sizes):
    return [
        {"id": i, "buy": float(buy_levels[i]), "sell": float(sell_levels[i]), "size": float(sizes[i]), "open": False}
        for i in range(len(buy_levels))
    ]

def run_grid_strategy_month_reentry(
    close_series,
    grid_spacing,
    n_levels=10,
    base_size=1.5,
    martingale_multiplier=2.0,
    use_buy_levels=8,
    take_profit_pct=0.025,
    stop_loss_index=9,
    max_cycles_per_month=None,
    allow_immediate_reentry=True
):
    """
    Monthly grid runner with re-entry after basket take profit.

    Behavior:
    - Build grid anchored at first price of close_series.
    - Trade intra-month; on basket TP: close all opens, rebuild grid at current price, continue.
    - Stop-loss is checked against current grid's buy_levels[stop_loss_index].
    - If max_cycles_per_month is set, caps the number of rebuilds per month.
    - If allow_immediate_reentry is True, the same price bar after rebuild may trigger immediate buys.
    """

    # defensive: empty series
    if not close_series:
        return {"realized_pnl": 0.0, "trades": 0, "tp_hits": 0, "sl_hit": False, "cycles_run": 0}

    pnl = 0.0
    trades = 0
    tp_hits = 0
    sl_hit = False
    cycles = 0

    i = 0
    L = len(close_series)

    # ---------- initialize first grid ----------
    ref_price = float(close_series[0])
    buy_levels, sell_levels = create_grid_levels(ref_price, grid_spacing, n_levels)
    sizes = compute_martingale_sizes(base_size, martingale_multiplier, n_levels)
    positions = initialize_grid_positions(buy_levels, sell_levels, sizes)
    cycles += 1

    # ensure stop_loss_index valid for the grid
    if stop_loss_index >= len(positions):
        stop_loss_index = len(positions) - 1
    if stop_loss_index < 0:
        stop_loss_index = 0

    # ---------- main loop ----------
    while i < L:
        price = float(close_series[i])

        # ---------- BUYS ----------
        for p in positions[:use_buy_levels]:
            if not p["open"] and price <= p["buy"]:
                p["open"] = True
                trades += 1

        # ---------- INDIVIDUAL GRID SELLS ----------
        for p in positions:
            if p["open"] and price >= p["sell"]:
                p["open"] = False
                pnl += (p["sell"] - p["buy"]) * p["size"]
                trades += 1

        # ---------- BASKET TAKE PROFIT ----------
        open_positions = [p for p in positions if p["open"]]
        avg = compute_average_price(open_positions)  # pass only open positions (defensive)

        if avg is not None and price >= avg * (1 + take_profit_pct):
            # close all open positions at market price
            for p in open_positions:
                p["open"] = False
                pnl += (price - p["buy"]) * p["size"]
                trades += 1

            tp_hits += 1

            # cycle cap check
            if max_cycles_per_month is not None and cycles >= max_cycles_per_month:
                # do not rebuild further, finish month
                break

            # ---------- REBUILD GRID anchored at current price ----------
            buy_levels, sell_levels = create_grid_levels(price, grid_spacing, n_levels)
            sizes = compute_martingale_sizes(base_size, martingale_multiplier, n_levels)
            positions = initialize_grid_positions(buy_levels, sell_levels, sizes)
            cycles += 1

            # re-validate stop_loss_index for new grid
            if stop_loss_index >= len(positions):
                stop_loss_index = len(positions) - 1
            if stop_loss_index < 0:
                stop_loss_index = 0

            # allow immediate buys on same price if requested
            if allow_immediate_reentry:
                # do not increment i, let loop re-process same price with new grid
                continue
            else:
                # proceed to next bar
                i += 1
                continue

        # ---------- STOP LOSS (based on current grid's buy level at stop_loss_index) ----------
        # Use the current grid's buy level (no buffer) as the SL trigger:
        current_stop_level = positions[stop_loss_index]["buy"]
        if price < current_stop_level:
            # close all open positions at market price and terminate month
            for p in positions:
                if p["open"]:
                    p["open"] = False
                    pnl += (price - p["buy"]) * p["size"]
                    trades += 1
            sl_hit = True
            break

        # move to next price bar
        i += 1

    return {
        "realized_pnl": pnl,
        "trades": trades,
        "tp_hits": tp_hits,
        "sl_hit": sl_hit,
        "cycles_run": cycles
    }


# build month windows 
month_starts = pd.date_range(start=BACKTEST_START, end=BACKTEST_END, freq="MS").to_pydatetime().tolist()

# storage for results 
records = []  # one row per (ticker, month, timeframe)

# ---------- sanity: require phase1 outputs ----------
if "spacing_meta" not in globals() or "main_dict" not in globals():
    raise SystemExit("spacing_meta or main_dict not found. Run Phase 1/1.5 first.")

# ---------- build month windows ----------
month_starts = pd.date_range(start=BACKTEST_START, end=BACKTEST_END, freq="MS").to_pydatetime().tolist()

# ---------- run: timeframe -> month -> ticker ----------
records = []  # collect (ticker, month, timeframe, metrics)

for tf in TIMEFRAMES:
    print(f"Running timeframe: {tf}")
    for i, start_dt in enumerate(month_starts):
        end_dt = (month_starts[i+1] - timedelta(seconds=1)) if i+1 < len(month_starts) else BACKTEST_END
        month_label = start_dt.strftime("%Y-%m")
        tested = 0

        for ticker, meta in spacing_meta.items():
            try:
                # get month data for this timeframe
                if tf == "1d":
                    df_all = main_dict.get(ticker)
                    if df_all is None:
                        continue
                    df_month = df_all.loc[(df_all["timestamp"] >= start_dt) & (df_all["timestamp"] <= end_dt)].copy()
                    used_tf = "1d"
                else:
                    # fetch intraday for the month (no fallback)
                    df_month = yf.Ticker(ticker).history(interval=tf, start=start_dt, end=end_dt + timedelta(days=1))
                    if df_month is None or df_month.empty:
                        continue
                    df_month = df_month.reset_index()
                    # normalize first datetime-like column to 'timestamp'
                    dt_col = df_month.columns[0]
                    df_month = df_month.rename(columns={dt_col: "timestamp"})
                    df_month["timestamp"] = pd.to_datetime(df_month["timestamp"])
                    used_tf = tf

                # require Close column and chronological order
                if "Close" not in df_month.columns or df_month.empty:
                    continue
                df_month = df_month.sort_values("timestamp").reset_index(drop=True)

                # grid anchored at first close of month
                ref_price = float(df_month["Close"].iloc[0])
                spacing = meta["grid_spacing"]

                buy_levels, sell_levels = create_grid_levels(ref_price, spacing, n_levels=N_LEVELS)
                sizes = compute_martingale_sizes(BASE_SIZE, MARTINGALE_MULTIPLIER, n_levels=N_LEVELS)

                close_series = df_month["Close"].tolist()
                metrics = run_grid_strategy_month_reentry(
                    close_series=close_series,
                    grid_spacing=spacing,        
                    n_levels=N_LEVELS,
                    base_size=BASE_SIZE,
                    martingale_multiplier=MARTINGALE_MULTIPLIER,
                    use_buy_levels=USE_BUY_LEVELS,
                    take_profit_pct=TAKE_PROFIT_PCT,
                    stop_loss_index=STOP_LOSS_INDEX,
                    max_cycles_per_month=None,    
                    allow_immediate_reentry=True
                )

                records.append({
                    "ticker": ticker,
                    "month": month_label,
                    "timeframe": used_tf,
                    "realized_pnl": metrics["realized_pnl"],
                    "trades": metrics["trades"],
                    "tp_hits": metrics["tp_hits"],
                    "sl_hit": metrics["sl_hit"],
                    "ref_price": ref_price,
                    "spacing": spacing
                })
                tested += 1

            except Exception as e:
                # minimal error reporting, continue others
                print(f"[{month_label}][{ticker}][{tf}] error: {e}")
                continue

        if tested:
            print(f"{month_label} | tested: {tested}")

# ---------- save results ----------
results_df = pd.DataFrame.from_records(records)
results_df.to_csv("phase3_monthly_results_minimal.csv", index=False)
print("Saved phase3_monthly_results.csv")

# quick aggregated summary per timeframe
if not results_df.empty:
    summary = results_df.groupby("timeframe").agg(
        mean_monthly_pnl = ("realized_pnl", "mean"),
        mean_monthly_trades = ("trades", "mean"),
        tp_rate = ("tp_hits", "mean"),
        sl_fraction = ("sl_hit", "mean")
    ).reset_index()
    print("\n=== Aggregated summary by timeframe ===")
    print(summary.to_string(index=False))
else:
    print("No results collected.")


Running timeframe: 1d
2025-01 | tested: 17
2025-02 | tested: 16
2025-03 | tested: 16
2025-04 | tested: 16
2025-05 | tested: 16
2025-06 | tested: 16
2025-07 | tested: 16
2025-08 | tested: 16
2025-09 | tested: 16
2025-10 | tested: 16
2025-11 | tested: 16
2025-12 | tested: 16
Running timeframe: 4h


$USDE-USD: possibly delisted; no price data found  (4h 2025-01-01 00:00:00+00:00 -> 2025-02-01 23:59:59+00:00)


2025-01 | tested: 16


$USDE-USD: possibly delisted; no price data found  (4h 2025-02-01 00:00:00+00:00 -> 2025-03-01 23:59:59+00:00)


2025-02 | tested: 16


...

2025-10 | tested: 16


$USDE-USD: possibly delisted; no price data found  (1h 2025-11-01 00:00:00+00:00 -> 2025-12-01 23:59:59+00:00)


2025-11 | tested: 16


$USDE-USD: possibly delisted; no price data found  (1h 2025-12-01 00:00:00+00:00 -> 2026-01-01 00:00:00+00:00)


2025-12 | tested: 16
Saved phase3_monthly_results.csv

=== Aggregated summary by timeframe ===
timeframe  mean_monthly_pnl  mean_monthly_trades  tp_rate  sl_fraction
       1d       1258.584599             2.186528 0.326425     0.020725
       1h       2097.005409             3.609375 0.635417     0.015625
       4h       1510.450905             3.135417 0.598958     0.015625
